***

# Revenue Forecast using ARIMA

*Posted on April 2022*

***

In this notebook Im going to try forecast revenue using ARIMA model

In [11]:
import warnings
import pandas as pd
import plotly.express as px

from pandas.plotting import autocorrelation_plot
from matplotlib import pyplot
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from sklearn.metrics import mean_squared_error
from math import sqrt

In [12]:
path_file = '/kaggle/input/time-series-starter-dataset/Month_Value_1.csv'

df = pd.read_csv(path_file)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/time-series-starter-dataset/Month_Value_1.csv'

In [ ]:
df.dtypes

In [ ]:
df['Period'] = pd.to_datetime(df['Period'], dayfirst=True)

In [13]:
df = df.dropna()

NameError: name 'df' is not defined

In [14]:
df.head()

NameError: name 'df' is not defined

In [15]:
fig = px.line(df, x='Period', y="Revenue")
fig.show()

NameError: name 'df' is not defined

In [16]:
series = pd.Series(df['Revenue'])
series.head()

NameError: name 'df' is not defined

## Autocorrelation Function and Partial Autocorrelation Function

In [8]:
autocorrelation_plot(series)
pyplot.show()

NameError: name 'autocorrelation_plot' is not defined

Looks like there are correlation at the first 10 lag

In [9]:
plot_acf(series, lags=10)
pyplot.show()

NameError: name 'plot_acf' is not defined

In [10]:
plot_pacf(series, lags=10)
pyplot.show()

NameError: name 'plot_pacf' is not defined

## Base model

Base model with the next prediction is the same as current observastion

In [12]:
X = series.values
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()

In [13]:

warnings.filterwarnings('ignore')
for t in range(len(test)):
    #model = ARIMA(history, order=(1,1,1))
    #model_fit = model.fit()
    #output = model_fit.forecast()
    yhat = test[t-1]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    
    print("predicted: {}, expected: {}".format(yhat, obs))
    
rmse = sqrt(mean_squared_error(test, predictions))
print("Test RMSE: {}".format(rmse))

predicted: 52320692.9428, expected: 35826534.9072
predicted: 35826534.9072, expected: 23268655.2112
predicted: 23268655.2112, expected: 35423489.85
predicted: 35423489.85, expected: 39831565.6974
predicted: 39831565.6974, expected: 32999145.2096
predicted: 32999145.2096, expected: 47221828.2018
predicted: 47221828.2018, expected: 36459960.091485
predicted: 36459960.091485, expected: 36546498.663015
predicted: 36546498.663015, expected: 54198706.7196
predicted: 54198706.7196, expected: 32743989.6056
predicted: 32743989.6056, expected: 32531657.5397
predicted: 32531657.5397, expected: 47709701.6346
predicted: 47709701.6346, expected: 45992141.57398
predicted: 45992141.57398, expected: 36933665.022
predicted: 36933665.022, expected: 48526260.1344
predicted: 48526260.1344, expected: 44160416.1824
predicted: 44160416.1824, expected: 36374956.4944
predicted: 36374956.4944, expected: 58756473.6608
predicted: 58756473.6608, expected: 56288300.87
predicted: 56288300.87, expected: 40225243.264
p

In [14]:
df_plot = pd.DataFrame(data={'Test': test, 'Base Model Predictions': predictions})

fig = px.line(df_plot, x=df_plot.index, y=df_plot.columns)
fig.show()

## Gridsearch

Next, Im going to use gridsearch to find the best parameters for the ARIMA model

In [15]:

def evaluate_arima(X, order):
    size = int(len(X) * 0.66)
    train, test = X[0:size], X[size:len(X)]
    history = [x for x in train]
    predictions = list()
    
    for t in range(len(test)):
        model = ARIMA(history, order=order)
        model_fit = model.fit()
        output = model_fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        obs = test[t]
        history.append(obs)
    
        print("predicted: {}, expected: {}".format(yhat, obs))
    
    rmse = sqrt(mean_squared_error(test, predictions))
    return rmse

def evaluate_model(dataset, ps, ds, qs):
    best_score, best_order = float('inf'), None
    
    for p in ps:
        for d in ds:
            for q in qs:
                
                order = (p,d,q)
                
                try:
                    rmse = evaluate_arima(dataset, order)
                    if rmse < best_score:
                        best_score, best_order = rmse, order
                
                    print('ARIMA {}, RMSE: {}'.format(rmse, order))
                except:
                    continue
                    
    print('Best ARIMA {}, RMSE: {}'.format(best_score, best_order))

In [16]:
ps = [0, 2, 4, 8, 10]
ds = [0, 2, 4, 8]
qs = [0, 2, 4, 8]

warnings.filterwarnings('ignore')
evaluate_model(series.values, ps, ds, qs)

predicted: 27302545.140351605, expected: 35826534.9072
predicted: 27500777.460510895, expected: 23268655.2112
predicted: 27404592.86393564, expected: 35423489.85
predicted: 27582790.574737072, expected: 39831565.6974
predicted: 27849068.294794958, expected: 32999145.2096
predicted: 27958644.39936528, expected: 47221828.2018
predicted: 28359960.728582677, expected: 36459960.091485
predicted: 28525266.83802965, expected: 36546498.663015
predicted: 28685691.47452936, expected: 54198706.7196
predicted: 29185946.675413094, expected: 32743989.6056
predicted: 29254370.57791669, expected: 32531657.5397
predicted: 29316206.180969212, expected: 47709701.6346
predicted: 29656826.467147563, expected: 45992141.57398
predicted: 29953832.196362693, expected: 36933665.022
predicted: 30078472.068249073, expected: 48526260.1344
predicted: 30402117.47291839, expected: 44160416.1824
predicted: 30639329.51963358, expected: 36374956.4944
predicted: 30736543.53615504, expected: 58756473.6608
predicted: 31203

## Training The Model

In [17]:
X = series.values
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()

In [18]:

warnings.filterwarnings('ignore')
for t in range(len(test)):
    model = ARIMA(history, order=(8,2,2))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    
    print("predicted: {}, expected: {}".format(yhat, obs))
    
rmse = sqrt(mean_squared_error(test, predictions))
print("Test RMSE: {}".format(rmse))

predicted: 31288275.50904497, expected: 35826534.9072
predicted: 26308731.21719458, expected: 23268655.2112
predicted: 36203648.46719566, expected: 35423489.85
predicted: 35149470.936555915, expected: 39831565.6974
predicted: 31434139.930569187, expected: 32999145.2096
predicted: 47784153.23662603, expected: 47221828.2018
predicted: 46599785.251406096, expected: 36459960.091485
predicted: 35109659.626998685, expected: 36546498.663015
predicted: 40409606.299696535, expected: 54198706.7196
predicted: 41739359.21995257, expected: 32743989.6056
predicted: 38143043.93211372, expected: 32531657.5397
predicted: 42138082.95161258, expected: 47709701.6346
predicted: 29759650.526685353, expected: 45992141.57398
predicted: 39156172.68260505, expected: 36933665.022
predicted: 50243450.854225166, expected: 48526260.1344
predicted: 40940609.34525506, expected: 44160416.1824
predicted: 36592676.11115191, expected: 36374956.4944
predicted: 51205866.32254361, expected: 58756473.6608
predicted: 47760942

In [1]:
df_plot = pd.DataFrame(data={'Test': test, 'Predictions': predictions})

fig = px.line(df_plot, x=df_plot.index, y=df_plot.columns)
fig.show()

NameError: name 'pd' is not defined

Reference:

Jason Brownlee - Introduction to Time Series Forecasting with Python, 2020